In [13]:
import torch
from torchtext import data
from torchtext import datasets
import random, re

In [24]:
def clean_data(string):
    """
     Argument:
             string: the string to be cleaned
    """
    string = str(string)
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r"\'m", " \'m", string)
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [25]:
file = "E:\Text_Classification\data\Yelp\yelp_review_full_csv"
TEXT = data.Field(tokenize='spacy',preprocessing=clean_data)
LABEL = data.LabelField(dtype=torch.int8)
tv_dataFields = [("label", LABEL), ("comment_text", TEXT)]
trn = data.TabularDataset.splits(path=file, train='test.csv',\
                                     format='csv',\
                                     skip_header=False, fields=tv_dataFields)

In [26]:
trn, val = trn[0].split(random_state=random.seed(1234))

In [27]:
val[0].__dict__

{'label': '3',
 'commet_text': "'they' , 'were' , 'solid' , ' ' , 'the' , 'po' , 'boy' , 'was' , 'good' , ' '"}

In [23]:
trn[0].__dict__

{'label': '1',
 'commet_text': ['I',
  "'m",
  'writing',
  'this',
  'review',
  'about',
  'our',
  'stay',
  '8/20',
  '-',
  '8/23',
  '.',
  'We',
  'chose',
  'to',
  'celebrate',
  'my',
  'husband',
  "'s",
  'birthday',
  'and',
  'our',
  'anniversary',
  'at',
  'the',
  'MO',
  '.',
  'We',
  'did',
  'enjoy',
  'the',
  'pool',
  'and',
  'spa',
  '.',
  '\\n\\nHOWEVER',
  ',',
  'on',
  'our',
  'last',
  'night',
  'a',
  'very',
  'young',
  ',',
  'loud',
  'group',
  'of',
  'party',
  'people',
  'checked',
  'in',
  'next',
  'door',
  '.',
  'At',
  'midnight',
  ',',
  'we',
  'called',
  'the',
  'front',
  'desk',
  'and',
  'asked',
  'if',
  'they',
  'could',
  'speak',
  'to',
  'this',
  'group',
  'and',
  'ask',
  'them',
  'to',
  'turn',
  'down',
  'the',
  'LOUD',
  'RAP',
  'MUSIC',
  'and',
  'STOP',
  'YELLING',
  '.',
  'The',
  'front',
  'desk',
  'was',
  'apologetic',
  'and',
  'called',
  'their',
  'room',
  'to',
  'ask',
  'them',
  'to',

In [9]:
trn[0].__dict__

{'label': '0',
 'commet_text': '[\'ok\', \',\', \'so\', \'i\', "\'ve", \'been\', \'here\', \'before\', \'3\', \'years\', \'ago\', \'was\', \'very\', \'impressed\', \'since\', \'it\', \'was\', \'my\', \'first\', \'time\', \'to\', \'las\', \'vegas\', \'i\', \'was\', \'not\', \'impressed\', \'with\', \'the\', \'check\', \'in\', \'staff\', \'the\', \'ladies\', \'who\', \'answer\', \'the\', \'phone\', \'are\', \'rude\', \'and\', \'short\', \'i\', "\'m", \'visiting\', \'vegas\', \'again\', \'next\', \'week\', \'and\', \'i\', \'booked\', \'an\', \'appointment\', \'with\', \'a\', \'lady\', \'who\', \'was\', \'a\', \'bit\', \'rude\', \'or\', \'not\', \'so\', \'friendly\', \'and\', \'when\', \'i\', \'called\', \'today\', \'to\', \'ask\', \'a\', \'question\', \'the\', \'lady\', \'was\', \'so\', \'unpleasant\', \'again\', \'so\', \'i\', \'just\', \'canceled\', \'my\', \'appointment\', \'all\', \'together\', \'customer\', \'service\', \'is\', \'very\', \'important\', \'in\', \'the\', \'salon\', \'a

In [2]:
SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [6]:
# <class 'torchtext.data.example.Example'>
# <torchtext.data.example.Example object at 0x00000294BFF96278>\
test_data[0].__dict__

{'text': ['I',
  'went',
  'and',
  'saw',
  'this',
  'movie',
  'last',
  'night',
  'after',
  'being',
  'coaxed',
  'to',
  'by',
  'a',
  'few',
  'friends',
  'of',
  'mine',
  '.',
  'I',
  "'ll",
  'admit',
  'that',
  'I',
  'was',
  'reluctant',
  'to',
  'see',
  'it',
  'because',
  'from',
  'what',
  'I',
  'knew',
  'of',
  'Ashton',
  'Kutcher',
  'he',
  'was',
  'only',
  'able',
  'to',
  'do',
  'comedy',
  '.',
  'I',
  'was',
  'wrong',
  '.',
  'Kutcher',
  'played',
  'the',
  'character',
  'of',
  'Jake',
  'Fischer',
  'very',
  'well',
  ',',
  'and',
  'Kevin',
  'Costner',
  'played',
  'Ben',
  'Randall',
  'with',
  'such',
  'professionalism',
  '.',
  'The',
  'sign',
  'of',
  'a',
  'good',
  'movie',
  'is',
  'that',
  'it',
  'can',
  'toy',
  'with',
  'our',
  'emotions',
  '.',
  'This',
  'one',
  'did',
  'exactly',
  'that',
  '.',
  'The',
  'entire',
  'theater',
  '(',
  'which',
  'was',
  'sold',
  'out',
  ')',
  'was',
  'overcome',


In [3]:
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train_data)

In [4]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, 
    device=device)

In [5]:
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs,embedding_dim)) for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [6]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [7]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1123,  0.3113,  0.3317,  ..., -0.4576,  0.6191,  0.5304],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [8]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [9]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [10]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [11]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [13]:
N_EPOCHS = 10


for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')
    filepath = 'decoder-{}.ckpt'.format(epoch + 1)
    torch.save(model.state_dict(), filepath)

| Epoch: 01 | Train Loss: 0.492 | Train Acc: 74.96% | Val. Loss: 0.340 | Val. Acc: 85.30% |
| Epoch: 02 | Train Loss: 0.296 | Train Acc: 87.83% | Val. Loss: 0.300 | Val. Acc: 87.35% |
| Epoch: 03 | Train Loss: 0.217 | Train Acc: 91.38% | Val. Loss: 0.271 | Val. Acc: 89.03% |
| Epoch: 04 | Train Loss: 0.143 | Train Acc: 94.81% | Val. Loss: 0.270 | Val. Acc: 89.32% |
| Epoch: 05 | Train Loss: 0.085 | Train Acc: 97.36% | Val. Loss: 0.289 | Val. Acc: 88.91% |
| Epoch: 06 | Train Loss: 0.048 | Train Acc: 98.61% | Val. Loss: 0.332 | Val. Acc: 88.63% |
| Epoch: 07 | Train Loss: 0.030 | Train Acc: 99.29% | Val. Loss: 0.368 | Val. Acc: 88.87% |
| Epoch: 08 | Train Loss: 0.017 | Train Acc: 99.70% | Val. Loss: 0.387 | Val. Acc: 88.90% |
| Epoch: 09 | Train Loss: 0.011 | Train Acc: 99.83% | Val. Loss: 0.434 | Val. Acc: 88.75% |
| Epoch: 10 | Train Loss: 0.009 | Train Acc: 99.84% | Val. Loss: 0.458 | Val. Acc: 88.58% |


In [29]:
model.load_state_dict(torch.load('decoder-5.ckpt'))

In [30]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

| Test Loss: 0.312 | Test Acc: 88.24% |


In [31]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(sentence, min_len=5):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [33]:
predict_sentiment("This film is great")

0.9371322393417358